# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,75.51,81,67,8.12,PN,1745425306
1,1,edinburgh of the seven seas,-37.0676,-12.3116,65.07,87,8,16.98,SH,1745425307
2,2,hilo,19.7297,-155.0900,70.50,85,40,3.00,US,1745425250
3,3,alo,6.5486,4.2686,85.06,72,97,6.80,NG,1745425309
4,4,walla walla,46.0646,-118.3430,52.86,59,0,8.05,US,1745425310


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM",
                                      frame_width=700, frame_height=500,
                                      size="Humidity", scale=1,
                                      color="City", hover_cols=["City", "Country", "Humidity"])
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Filter for ideal vacation weather conditions
vacation_criteria_df = city_data_df[
    (city_data_df["Max Temp"] > 70) &
    (city_data_df["Max Temp"] < 85) &
    (city_data_df["Wind Speed"] < 10) &
    (city_data_df["Cloudiness"] < 10)
]

# Drop any rows with null values
vacation_criteria_df = vacation_criteria_df.dropna()

# Display sample data
vacation_criteria_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,tura,25.5198,90.2201,76.39,83,0,9.15,IN,1745425319
24,24,la'ie,21.6477,-157.9253,72.79,90,6,1.99,US,1745425334
41,41,fale old settlement,-9.3852,-171.2468,82.99,76,6,5.59,TK,1745425355
57,57,cidreira,-30.1811,-50.2056,78.55,65,2,6.76,BR,1745425373
63,63,hamilton,39.1834,-84.5333,75.42,40,0,6.91,US,1745425084


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_criteria_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
11,tura,IN,25.5198,90.2201,83,
24,la'ie,US,21.6477,-157.9253,90,
41,fale old settlement,TK,-9.3852,-171.2468,76,
57,cidreira,BR,-30.1811,-50.2056,65,
63,hamilton,US,39.1834,-84.5333,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 1000
params = {
  "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    base_url = "https://api.geoapify.com/v2/places"

    try:
        response = requests.get(base_url, params=params)
        name_address = response.json()
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
tura - nearest hotel: No hotel found
la'ie - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
cidreira - nearest hotel: Hotel Castelo
hamilton - nearest hotel: No hotel found
tecozautla - nearest hotel: La Terracita


,City,Country,Lat,Lng,Humidity,Hotel Name
11,tura,IN,25.5198,90.2201,83,No hotel found
24,la'ie,US,21.6477,-157.9253,90,No hotel found
41,fale old settlement,TK,-9.3852,-171.2468,76,No hotel found
57,cidreira,BR,-30.1811,-50.2056,65,Hotel Castelo
63,hamilton,US,39.1834,-84.5333,40,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the final map plot
hotel_map_plot = hotel_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM",
                                        frame_width=700, frame_height=500,
                                        size="Humidity", scale=1,
                                        color="Hotel Name", hover_cols=["City", "Country", "Hotel Name", "Humidity"])

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City,Country)